In [9]:
from utils.helper import read_xml, descompress_zip, count_ingredients
import os
import pandas as pd

In [10]:
data_path = os.path.join("..", "data")

In [3]:
coctails = read_xml(os.path.join(data_path, "ccc_cocktails.xml"))
print(len(coctails))
print(type(coctails))

108
<class 'xml.etree.ElementTree.Element'>


In [11]:
df = pd.read_csv(os.path.join(data_path, "recipes_w_search_terms.csv"))
n, ingredients = count_ingredients(df["ingredients"])
print(f"Ingredients: \n {ingredients}")
print(f"Number of ingredients: \n {n}")

Ingredients: 
 ['chicken bones', 'poblano pepper', 'roasted sunflower seeds', 'frangelico liqueur', 'low-fat salsa', 'salt %26 pepper', 'thai rice', 'mutton shoulder', 'chunk tuna', 'plums in light syrup', 'dark turkey meat', 'dried tart cherries', 'cut yams in syrup', 'rock sugar', 'light lemonade', 'wholemeal self-rising flour', 'blackberry jam', 'fat-free ham', 'barbecued beef', 'dried ghost chile', 'raspberry pastry filling', 'condensed vegetable broth', 'sherry vinaigrette', 'lowfat curds', 'liquid gravy browner', 'lavender flowers', 'cassava meal', 'raw aku', 'unsweetened kool-aid powdered drink mix', 'freshly milled black pepper', 'unsalted creamy peanut butter', 'puffed quinoa', 'lemon essence', 'chocolate ice cream cones', 'cauliflower', 'cinnamon doughnut holes', 'chili sauce', 'bamboo skewer', 'leaf lettuce', 'pepperidge farm mint milanos', 'fresh swiss chard', 'salad elegance seasoning mix', 'meat', 'cornbread stuffing mix', 'sweet whole kernel corn', 'toasted pine nut cous